----
## Парсер для сайта amazon.com
----

Этот парсер предназначен для сохранения отзывов к одному конкретному товару


Сначала создадим список всех адресов, к которым будем обращаться 



In [108]:
base_url = 'https://www.amazon.es/CREATE-THERA-Cafetera-monodosis-semiautom%C3%A1tica/product-reviews/B0BSH8CYN8/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&pageNumber=1&reviewerType=avp_only_reviews'
urls = []

for i in range(1, 319):
    
    prefix = base_url[:155]  
    suffix = base_url[156:]

    # Формируем новый URL с обновленным номером страницы
    updated_url = f"{prefix}{i}{suffix}"
    
    urls.append(updated_url)


In [78]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd

import time
import random
from tqdm import tqdm

def scrape_reviews(urls):
    
    options = Options()
    options.add_argument('--headless') 
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")


    DRIVER_PATH = '/Users/apple/Downloads/Project_Gnomi_Huekradi/chromedriver'

    service = Service(DRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=options)
    
    reviews = []
    
    c = 0
    for url in tqdm(urls, desc='Processing URLs', unit='URL'): 
        
        c += 1
        driver.get(url)
        
        if c % 5 == 0:
            time.sleep(10)
        else:
            time.sleep(random.randint(1, 3))


        # Getting reviews
        review_blocks = driver.find_elements(By.CSS_SELECTOR, '.a-section.review')
        for review_block in review_blocks:
            title = review_block.find_element(By.CSS_SELECTOR, '.review-title-content').text.strip()
            rating = review_block.find_element(By.CSS_SELECTOR, '.a-icon-alt').get_attribute('textContent').strip()
            body = review_block.find_element(By.CSS_SELECTOR, '[data-hook="review-body"]').text.strip()
            author = review_block.find_element(By.CSS_SELECTOR, '.a-profile-name').text.strip()
            date = review_block.find_element(By.CSS_SELECTOR, '.review-date').text.strip()

            reviews.append({
                'title': title,
                'rating': rating,
                'body': body,
                'author': author,
                'date': date,
            })
    
    print("Done.")
    driver.quit()
    return reviews

# Функция для сохранения данных в CSV файл
def save_to_csv(reviews, filename='amazon_reviews.csv'):
    df = pd.DataFrame(reviews)
    df.to_csv(filename, index=False, encoding='utf-8')

if __name__ == "__main__":
    time.sleep(random.randint(1, 3))
    reviews = scrape_reviews(urls)
    save_to_csv(reviews, 'amazon_reviews.csv')
    print(f'Сохранено {len(reviews)} отзывов в amazon_reviews.csv')


Attempt  1
Attempt  2
Сохранено 20 отзывов в amazon_reviews.csv


In [113]:
# Saving data in DataFrame
data = pd.read_csv('amazon_reviews.csv')

data.head()

,title,rating,body,author,date
0,Perfecta!,"5,0 de 5 estrellas",Me encanta! Soy adicta al café y me gusta toma...,Perfecta!,Revisado en España el 22 de febrero de 2024
1,Un quiero y no puedo,"4,0 de 5 estrellas",Realmente el producto en si funciona bien pero...,Ro,Revisado en España el 17 de abril de 2024
2,Precioso diseño y buen funcionamiento a buen p...,"5,0 de 5 estrellas","Si quieres una cafetera sin complicaciones, bo...",Hank Solo,Revisado en España el 22 de enero de 2024
3,Buena Relación Precio/Calidad,"5,0 de 5 estrellas",Qué Pasada!!\n\nla Compramos para Reyes cansad...,LHO,Revisado en España el 26 de febrero de 2021
4,"Decepcionante, tanto la cafetera como el servi...","1,0 de 5 estrellas","EDITO: El café lo hace bueno, no sabe para nad...",Alicia TM,Revisado en España el 12 de diciembre de 2020


In [70]:
data.shape

(20, 5)

!!! Тут старая версия. Палкой не тыкать !!!

In [61]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

def scrape_reviews(url):
    # Настройки Chrome WebDriver
    options = Options()
    options.add_argument('--headless')  # Запуск без GUI
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    DRIVER_PATH = '/Users/apple/Downloads/Project_Gnomi_Huekradi/chromedriver'  

    # Запуск WebDriver
    service = Service(DRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    time.sleep(3)  # Подождать некоторое время для загрузки страницы

    reviews = []

    while True:
        try:
            # Извлечение отзывов
            review_blocks = driver.find_elements_by_css_selector('.review')
            for review_block in review_blocks:
                title = review_block.find_element_by_css_selector('.review-title').text.strip()
                rating = review_block.find_element_by_css_selector('.review-rating').text.strip()
                body = review_block.find_element_by_css_selector('.review-text').text.strip()
                author = review_block.find_element_by_css_selector('.a-profile-name').text.strip()
                date = review_block.find_element_by_css_selector('.review-date').text.strip()

                reviews.append({
                    'title': title,
                    'rating': rating,
                    'body': body,
                    'author': author,
                    'date': date,
                })

            # Переход на следующую страницу с отзывами, если такая есть
            next_button = driver.find_element_by_css_selector('.a-last > a')
            next_page_url = next_button.get_attribute('href')
            driver.get(next_page_url)
            time.sleep(3)  # Подождать загрузки новой страницы

        except Exception as e:
            print(f"Не удалось перейти на следующую страницу: {e}")
            break

    driver.quit()  # Закрыть WebDriver
    return reviews

# Функция для сохранения данных в CSV файл
def save_to_csv(reviews, filename='amazon_reviews.csv'):
    df = pd.DataFrame(reviews)
    df.to_csv(filename, index=False, encoding='utf-8')

if __name__ == "__main__":
    url = 'https://www.amazon.es/CREATE-THERA-Cafetera-monodosis-semiautom%C3%A1tica/product-reviews/B0BSH8CYN8?ie=UTF8&reviewerType=avp_only_reviews'
    №url = 'https://www.amazon.es/CREATE-THERA-Cafetera-monodosis-semiautom%C3%A1tica/product-reviews/B0BSH8CYN8?ie=UTF8&pageNumber=1&reviewerType=avp_only_reviews&pageSize=10&claim_type=EmailAddress&new_account=1&'
    reviews = scrape_reviews(url)
    save_to_csv(reviews, 'amazon_reviews.csv')
    print(f'Сохранено {len(reviews)} отзывов в amazon_reviews.csv')


Не удалось перейти на следующую страницу: 'WebDriver' object has no attribute 'find_elements_by_css_selector'
Сохранено 0 отзывов в amazon_reviews.csv
